In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
import requests
import bs4 as bs
import time
import itertools

In [4]:
#https://api.geekdo.com/xmlapi/boardgame/174430

In [5]:
#bggid = ','.join(str(i) for i in range(40000, 40200))
#print(bggid)

In [105]:
# Create list of unique features based on tags chosen
def feature_list(feature_list, feature_name):
    feature = file.find_all(feature_name)
    for i in feature:
        if i.string in feature_list:
            continue
        else:
            feature_list.append(i.string)
            
    return feature_list

# get features/attributes from xml data
def feature_extract(dataframe, tag, attribute):
    #df_list_features = []
    columns = []
    data = []
    for i in boardgame.find_all(tag, type=attribute):
        columns.append(i['value'])
        data.append(1)
    #print(columns)
    #print(data)
    df = pd.DataFrame(data=[data], columns=columns, index=[int(boardgame['id'])])
    #df_list_features.append(df)
    #print(df.head())
    
    if not df.empty:
        #data = pd.concat(df_list_features, ignore_index=False, sort=False)
        dataframe = pd.concat([df, dataframe], ignore_index=False, sort=False)
        #print(dataframe.head())
        #print(df.head())
        #print(dataframe.head())
    else:
        #if there are no tags add an empty row to the dataframe
        df = pd.DataFrame(data=[[0]], columns=['dummy_column'], index=[int(boardgame['id'])])
        dataframe = pd.concat([df, dataframe], ignore_index=False, sort=False)
        
    return dataframe
        
def df_clean(dataframe):      
    dtype = {col: 'int64' for col in dataframe.columns}
    dataframe.fillna(value=0, inplace=True)
    dataframe = dataframe.astype(dtype)

    
    
def comment_threshold(threshold, boardgame, dictionary):
    # get the total number of comments for the boardgame
    
    comments = int(boardgame.find('comments')['totalitems'])
    # we will not use game if comments below threshold
    if comments < threshold:
        return 'under threshold'

    pull_count = (comments//100 +1)

    #if pull 1 or less we will get all the comments on the first pass through
    if pull_count > 1:
        dictionary[int(boardgame['id'])] = pull_count
        
    return

In [140]:
batch_limit = 500
#comment threshold
threshold = 20
total_records = 350000

cat_list=[]
mech_list =[]


#commenting these out due to split extraction
comment_count = {}
cat_df = pd.DataFrame()
fam_df = pd.DataFrame()
mech_df = pd.DataFrame()
user_df = pd.DataFrame()

# pull data from XML API in batches
for batch in range(0, (total_records//batch_limit)+1):
#for batch in range(0, 1):
#for batch in range(0, 2):
    rating_df = pd.DataFrame()
    print(batch)
    bggid = ','.join(str(i) for i in range(batch_limit*(batch), batch_limit*(batch+1)))
    #api_url = "https://boardgamegeek.com/xmlapi2/thing?id=174430,174431&type=boardgame&stats=1&ratingcomments=1"
    #board games
    api_url="https://api.geekdo.com/xmlapi2/thing?id=" +str(bggid) + str("&type=boardgame&stats=1&ratingcomments=1")
    #board game expansions
    #api_url="https://api.geekdo.com/xmlapi2/thing?id=" +str(bggid) + str("&type=boardgameexpansion&stats=1&ratingcomments=1")
    try:
        response=requests.get(api_url)
    except:
        print('chunk error?')
        time.sleep(300)
        response=requests.get(api_url)
        
    if response.status_code == 200:
        file = bs.BeautifulSoup(response.text, 'lxml')
    else:
        max_retries = 3
        retry_count = 0
        success = False
        while retry_count < max_retries:
            time.sleep(30)
            response=requests.get(api_url)
            if response.status_code == 200:
                file = bs.BeautifulSoup(response.text, 'lxml')
                success = True
                break
            else:
                retry_count += 1
        if not success:
            print('API request failed after {} retries'.format(max_retries))

    # collect all top level XML boardgame tags
    boardgames = file.find_all('item', type='boardgame')

    # Add new features to cat_list and mech_list for DataFrame keys
    #cat_list = feature_list(cat_list, 'boardgamecategory')
    #mech_list = feature_list(mech_list, 'boardgamemechanic')
    # Create temporary mechanics and categories DataFrames
    cat_df_temp = pd.DataFrame(columns=cat_list)
    mech_df_temp = pd.DataFrame(columns=mech_list)
    fam_df_temp = pd.DataFrame()

    primary_element = {}
    secondary_element = {}
    # Define the elements you want to find
    elements = ['minplayers', 'maxplayers', 'minplaytime', 'maxplaytime', 'playingtime', 
                'yearpublished', 'minage', 'thumbnail', 'averageweight', 'description', 'bayesaverage',
               'average', 'usersrated']

    features = ['bggid', 'name']

    for feature_name in features:
        primary_element[feature_name] = []

    for element_name in elements:
        secondary_element[element_name] = []

    df_list = []
    
    for boardgame in boardgames:

        thumbnail_element = boardgame.find('thumbnail')
        comments_check = boardgame.find('comments')
            

        if thumbnail_element and comments_check:
            
            comment_check = comment_threshold(threshold, boardgame, comment_count_exp)
            if comment_check == 'under threshold':
                continue
            
            #primary_element['thumbnail'].append(thumbnail_element)
            primary_element['bggid'].append((int(boardgame['id'])))
            primary_element['name'].append(boardgame.find('name', type='primary')['value'])
            name = boardgame.find('name', type='primary')['value']


            for element_name in elements:
                try:
                    value = boardgame.find(element_name)[value]
                except:
                    value = boardgame.find(element_name).text
                if len(value) == 0:
                    value = 0
                    secondary_element[element_name].append(value)
                else:
                    secondary_element[element_name].append(value)
                  
            cat_df_temp = feature_extract(cat_df_temp, 'link', 'boardgamecategory')
            fam_df_temp = feature_extract(fam_df_temp, 'link', 'boardgamefamily')
            mech_df_temp = feature_extract(mech_df_temp, 'link', 'boardgamemechanic')
                
            comments = boardgame.find_all('comment')
            ratings = {'bggid': [int(boardgame['id'])] * len(comments), 
                       'username': [comment['username'] for comment in comments], 
                       'rating': [comment['rating'] for comment in comments]}
            rating_df_temp = pd.DataFrame.from_dict(ratings)

            df_list.append(rating_df_temp)
            #rating_df = pd.concat([rating_df, rating_df_temp], ignore_index=True, sort=False)
            
            
        else:
            continue
            
    # for sparse sections of BGGID where very few comments exist and we may return no boardgames
    if len(df_list) == 0:
        continue
    
    #print(mech_df_temp.head())
    cat_df = pd.concat([cat_df_temp, cat_df], ignore_index=False, sort=True)
    fam_df = pd.concat([fam_df_temp, fam_df], ignore_index=False, sort=True)
    #print(cat_df.head())
    mech_df = pd.concat([mech_df, mech_df_temp], ignore_index=False, sort=False)
    # Concat list of dfs before combining with existing user_df dataFrame
    comment_df = pd.concat(df_list, ignore_index=True, sort=True)
    user_df = pd.concat([user_df, comment_df], ignore_index=True, sort=True)
    
    if batch%50 == 0:
        print(user_df.info())
    
    
    # merge dictionaries, then convert to DataFrame and set data types                 
    game_elements = primary_element|secondary_element
    game_info = pd.DataFrame.from_dict(game_elements)
    game_info = game_info.astype({'bggid': 'int64', 'name': 'string', 'minplayers': 'int64', 'maxplayers': 'int64',
                          'minplaytime': 'int64', 'maxplaytime': 'int64', 'playingtime': 'int64', 'yearpublished': 'int64',
                         'minage': 'int64', 'thumbnail': 'string', 'averageweight': 'float64', 'description': 'string',
                         'bayesaverage': 'float64', 'average': 'float64', 'usersrated': 'float64'})
    
    # Replace table starting with the first batch
    if batch == 0:
        # Define database connection
        db_url = "connection_string"
        engine = create_engine(db_url)

        # Append the DataFrame to the existing table (if the column names and types match)
        game_info.to_sql('game_info_db', engine, if_exists='replace', index=False)
                     
    else:
        # Define database connection
        db_url = "connection_string"
        engine = create_engine(db_url)

        # Append the DataFrame to the existing table (if the column names and types match)
        game_info.to_sql('game_info_db', engine, if_exists='append', index=False)
    
    # set delay to not overwhelm server
    time.sleep(5)
                     
#print(game_elements)        

0


C:\Users\tdkla\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41596 entries, 0 to 41595
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   bggid     41596 non-null  int64 
 1   rating    41596 non-null  object
 2   username  41596 non-null  object
dtypes: int64(1), object(2)
memory usage: 975.0+ KB
None
1


C:\Users\tdkla\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


2


...

699


C:\Users\tdkla\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


700


C:\Users\tdkla\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097280 entries, 0 to 2097279
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   bggid     int64 
 1   rating    object
 2   username  object
dtypes: int64(1), object(2)
memory usage: 48.0+ MB
None


In [4]:
# Sort comment_count to maximize pull efficiency
comm_sorted = dict(sorted(comment_count.items(), key=lambda x:x[1], reverse=True))
user_df_copy = user_df.copy(deep=True)

In [9]:
def comment_pull(games_list, page):
    #*********************************************************************************************************
    # games_list: list of games to be entered into XML API URI as comma-separated list (setting max number at 500)
    # page: page to be entered into the XML API URI which dictates which group of 100 comments are returned
    # comments: dictionary with keys=bggid and values=(number of comments per game)
    #*********************************************************************************************************
    api_url="https://api.geekdo.com/xmlapi2/thing?id=" +str(games_list) + str("&type=boardgame&ratingcomments=1&page=") + str(page)
    print(page)
    #print(api_url)
    try:
        response=requests.get(api_url)
    except:
        print('chunk error?')
        time.sleep(120)
        response=requests.get(api_url)
    if response.status_code == 200:
        file = bs.BeautifulSoup(response.text, 'lxml')
    else:
        max_retries = 3
        retry_count = 0
        success = False
        while retry_count < max_retries:
            print('wait')
            time.sleep(30)
            response=requests.get(api_url)
            if response.status_code == 200:
                file = bs.BeautifulSoup(response.text, 'lxml')
                success = True
                break
            else:
                retry_count += 1
        if not success:
            print('API request failed after {} retries'.format(max_retries))

    # collect all top level XML boardgame tags
    boardgames = file.find_all('item', type='boardgame')
    
    df_list = []
    
    for boardgame in boardgames:
        comments = boardgame.find_all('comment')
        ratings = {'bggid': [int(boardgame['id'])] * len(comments), 
                   'username': [comment['username'] for comment in comments], 
                   'rating': [comment['rating'] for comment in comments]}
        rating_df_temp = pd.DataFrame.from_dict(ratings)
        #rating_df_temp = pd.concat([rating_df_temp, pd.DataFrame.from_dict(ratings)], ignore_index=True, sort=False)
        #print(rating_df_temp.head())
        df_list.append(rating_df_temp)
    batch_df = pd.concat(df_list, ignore_index=True, sort=False)
    return batch_df

    
def batched(iterable, n):
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(itertools.islice(it, n)):
        yield batch

batched_df = []
for batch in batched(comm_sorted, 50):
    # get number of pulls needed in batch from first game
    pull_count = comm_sorted[batch[0]]
    #print(pull_count)
    bggid = ','.join(str(i) for i in batch)
    page = 0
    rating_dfs = []
    # start at 1, because we already pulled once through the initial request
    for k in range(1, pull_count):
        #if k%15 == 0:
            #print(len(rating_dfs))
            
        rating_dfs.append(comment_pull(bggid, k+1))
        time.sleep(5)
        
    batched_df.append(pd.concat(rating_dfs))
        

#flattened_df_list = [df for sublist in rating_dfs for df in sublist]
rating_df = pd.concat(batched_df, ignore_index=True, sort=False)
user_df_copy = pd.concat([user_df_copy, rating_df], ignore_index=True, sort=True)

           
        
#print(batched(comm_sorted, 500))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
chunk error?
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
chunk error?
182
183
184
185
chunk error?
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268

120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
16

4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
4
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [10]:
user_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23698355 entries, 0 to 23698354
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   bggid     float64
 1   rating    object 
 2   username  object 
dtypes: float64(1), object(2)
memory usage: 542.4+ MB


In [17]:
# clean up dataframes
#df_clean(cat_df)
#df_clean(mech_df)
#df_clean(fam_df)

user_df_copy['rating'] = user_df_copy['rating'].replace('N/A', np.nan)
user_df_copy = user_df_copy.astype({'bggid': 'int64', 'username': 'string', 'rating': 'float64'})
user_df_copy.dropna(axis=0, inplace=True)

# Define database connection
db_url = "connection_string"
engine = create_engine(db_url)

# Append the DataFrame to the existing table (if the column names and types match)
#mech_df.to_sql('mechanics_table', engine, if_exists='replace', index=True)
#cat_df.to_sql('category_table', engine, if_exists='replace', index=True)
user_df_copy.to_sql('user_table', engine, if_exists='replace', index=False)

355

In [ ]:
'''for key in game_elements:
    print(key)
    print(len(game_elements[key]))'''

In [ ]:
#test = user_df[user_df['username'] == '4Corners']
#test.info()
#user_df.info()

In [ ]:
#test = file.find('item', type=='boardgame')

#gameid = test.find('comments')['totalitems']
#print(gameid)

In [128]:
print(comment_count)

{246015: 2, 246146: 3, 246150: 11, 246192: 121, 246200: 24, 246201: 11, 246203: 2, 246228: 11, 246263: 2, 246297: 20, 246315: 2, 246316: 3, 246355: 2, 246366: 3, 246508: 2, 246530: 4, 246531: 3, 246532: 3, 246534: 2, 246535: 4, 246567: 6, 246588: 2, 246639: 11, 246663: 21, 246684: 59, 246685: 2, 246689: 3, 246693: 2, 246696: 3, 246700: 2, 246701: 13, 246742: 3, 246744: 3, 246746: 2, 246757: 2, 246759: 10, 246761: 29, 246784: 137, 246812: 3, 246816: 10, 246855: 8, 246862: 2, 246873: 2, 246900: 121, 246912: 9, 246938: 4, 246960: 4, 247029: 2, 247034: 2, 247104: 8, 247106: 5, 247136: 2, 247143: 4, 247160: 15, 247182: 4, 247188: 3, 247191: 3, 247199: 2, 247236: 34, 247294: 2, 247313: 4, 247314: 10, 247324: 4, 247342: 11, 247360: 7, 247367: 60, 247394: 2, 247417: 8, 247436: 12, 247567: 3, 247578: 2, 247585: 11, 247643: 2, 247694: 10, 247704: 5, 247724: 5, 247733: 4, 247763: 185, 247778: 2, 247785: 4, 247935: 4, 247963: 2, 247980: 6, 247984: 3, 248002: 9, 248005: 17, 248065: 25, 248072: 2, 2

In [19]:
print(file)

<?xml version="1.0" encoding="utf-8"?><html><body><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="350000" type="boardgame">
<thumbnail>https://cf.geekdo-images.com/19n6ymIK2KLh5gSwAI0LXQ__thumb/img/0NJF0Wltqb-Cs3D7QfS4705Mb8o=/fit-in/200x150/filters:strip_icc()/pic6475965.png</thumbnail>
<image>https://cf.geekdo-images.com/19n6ymIK2KLh5gSwAI0LXQ__original/img/ZmtA4ahsUNCRc5dlD8BbGgAUlGc=/0x0/filters:format(png)/pic6475965.png</image>
<name sortindex="1" type="primary" value="Blue J's Card Game"></name>
<description>Blue J&amp;rsquo;s is a strategy game where the game grows into any shape as the cards are played out. Matching colors and/or symbols and placing them on the Start card is how the game begins. You force the other player(s) to draw cards based on how many complete squares you match. If you run out of cards you win the round or go for the quick win and match the Blue J&amp;rsquo;s. If you collect the most of nine Blue J&amp;rsquo;s Round Winner Tokens

In [88]:
user_df_double['rating'] = user_df_double['rating'].astype('float')
user_df_double['bggid'] = user_df_double['bggid'].astype('string')
user_df_double.describe(include='string')

,bggid
count,23698355
unique,77881
top,13.0
freq,123354


In [119]:
fam_df = fam_df.sort_index(axis=0)
fam_df.head()

,Admin: Better Description Needed!,Admin: Game System Entries,Ancient: Babylon,Ancient: Carthage,Ancient: Egypt,Ancient: Greece,Ancient: Mesopotamia,Ancient: Rome,Animals: Alligators / Crocodiles,Animals: Ants,...,Traditional Games: Tafl,Trivia: Movies,Versions & Editions: Board Game Versions of Role-Playing Games,Versions & Editions: Card Versions of Non-Card Games,Video Game Theme: Nintendo,Video Game Theme: Super Mario Bros.,Word Games: First Letter Given,Word Games: Guess the Word,Word Games: Spelling / Letters,dummy_column
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# clean up dataframes
df_clean(cat_df)
df_clean(mech_df)
df_clean(fam_df)

In [118]:
print(df_list)

[    bggid       username rating
0     500           fake      9
1     500  RichardIngram      9
2     500         kgnunn      8
3     500      moxtaveto      8
4     500           Sack      8
..    ...            ...    ...
57    500         tardor      4
58    500       skrutsch    3.5
59    500     bordspeler      3
60    500        Bankler      3
61    500      Maja Dorn      2

[62 rows x 3 columns],     bggid          username rating
0     503            lucsax     10
1     503       Capn_Future     10
2     503             zelig     10
3     503          cfarrell     10
4     503           njsauer     10
..    ...               ...    ...
95    503          greengow     10
96    503  SamwiseGamgeeBan     10
97    503             kathn     10
98    503      maddiedogson     10
99    503      darrenbailey     10

[100 rows x 3 columns],     bggid            username rating
0     504           Genre_Fad     10
1     504            joelbear     10
2     504           goserelin     1

In [141]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28970 entries, 350469 to 1
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Abstract Strategy           2129 non-null   float64
 1   Action / Dexterity          1476 non-null   float64
 2   Adventure                   1541 non-null   float64
 3   Age of Reason               222 non-null    float64
 4   American Civil War          300 non-null    float64
 5   American Indian Wars        69 non-null     float64
 6   American Revolutionary War  95 non-null     float64
 7   American West               302 non-null    float64
 8   Ancient                     905 non-null    float64
 9   Animals                     1865 non-null   float64
 10  Arabian                     85 non-null     float64
 11  Aviation / Flight           361 non-null    float64
 12  Bluffing                    1525 non-null   float64
 13  Book                        31